In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx

In [9]:
qvalue_cutoff = 0.0001

In [10]:
tomtom_df = pd.read_csv('tomtom_result.tsv', sep='\t', index_col=[0, 1])
tomtom_df = tomtom_df[tomtom_df['q-value'] < qvalue_cutoff]
print(tomtom_df.shape)

edges = set()
for s, t in tomtom_df.index:
    edges.add(tuple(sorted((s, t))))
print(len(edges), 'edges')

tomtom_df.head()

(6198, 8)
3908 edges


Optimal_offset       p-value       E-value       q-value  \
Query_ID Target_ID                                                             
MA0006.1 MA0006.1              0.0  9.209610e-15  6.667760e-12  1.298940e-11   
MA0854.1 MA0854.1              0.0  0.000000e+00  0.000000e+00  0.000000e+00   
         MA0853.1              0.0  1.668170e-23  1.207750e-20  1.011380e-20   
         MA0874.1              0.0  7.370820e-13  5.336470e-10  2.234400e-10   
         MA0628.1             -3.0  2.559770e-08  1.853270e-05  3.879860e-06   

                    Overlap    Query_consensus   Target_consensus Orientation  
Query_ID Target_ID                                                             
MA0006.1 MA0006.1       6.0             TGCGTG             TGCGTG           +  
MA0854.1 MA0854.1      17.0  CGAATTAATTAATCACC  CGAATTAATTAATCACC           +  
         MA0853.1      17.0  CGAATTAATTAATCACC  CGCATTAATTAATTACC           +  
         MA0874.1      17.0  CGAATTAATTAATCACC  TCCATTAATTAATGGAC           -  
         MA0628.1      10.0  CGAATTAATTAATCACC         ATTAATTAAT           +

In [11]:
g = nx.Graph()
g.add_edges_from(list(edges))
sub_graphs = nx.connected_component_subgraphs(g)

tf_to_group_id = {}
for i, sub_g in enumerate(sub_graphs):
    for node in sub_g.nodes:
        tf_to_group_id[node] = f'MotifGroup{i}'

In [12]:
meme_anno_df = pd.read_csv(
    '/home/hanliu/ref/MEME/selected_motifs_sets/JASPAR2020_CORE_vertebrates_non-redundant.mouse_genes.csv',
    index_col=0
)
meme_anno_df['motif_group'] = meme_anno_df.index.map(tf_to_group_id)

In [15]:
n_group = meme_anno_df['motif_group'].unique().size
n_group

199

In [14]:
meme_anno_df.to_csv(f'JASPAR2020_CORE_vertebrates_non-redundant.mouse_genes.with_motif_group.{n_group}.csv')